网络的设计模式:
- 残差连接
- 批标准化
- 深度可分离卷积

批标准化: 在训练过程中, 适应性的将数据标准化
批标准化原理: 训练过程中在内部保存已读取每批数据均值和方差的指数移动平均值
批标准化效果: 有助于梯度传播, 允许更深层的网络.
```python
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())
```

深度可分离卷积: 对输入的每个通道分别执行空间卷积, 然后通过逐点卷积(1x1)卷积将输出通道混合. 这相当于将空间特征和通道特征学习分开. 如果假设输入中的空间位置高度相关, 但不同的通道之间相对独立, 那么这么做时很有意义的.  
它需要的参数要少很多, 计算量也更小, 因此能够得到更小更快的模型.  
因为是一种执行卷积更高效的方法, 所以往往能够利用更少的数据学到更好的表示, 从而得到性能更好的模型.

In [ ]:
# 用于图像分类任务的轻量的深度可分离卷积神经网络

from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10
model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                 activation='relu',
                                 input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalMaxPooling2D())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')          

### 超参数优化
- 网络堆叠多少层?
- 每层包含多少个单元或过滤器?
- 激活函数使用relu还是其它函数?
- 是否应该使用BatchNormalization?
- 使用多大的dropout比率?
- ...
制定一个原则, 系统性的自动探索可能的决策空间. 根据经验搜索整个空间, 找出最佳的网络架构.超参数的优化过程通常如下所是:
- 选择一组超参数(自动选择)
- 构建相应的模型
- 将模型在训练数据上拟合, 并衡量其在验证数据上的性能.
- 选择要尝试的下一组超参数.
- 重复上述过程.
- 衡量模型在测试数据上的性能  
上述过程的关键在于, 给定许多组超参数, 使用验证性能的历史来选择下一组需要评估的超参数的算法. 有多种不同的技术可供选择: 贝叶斯优化, 遗传算法, 简单随机搜索.  
训练模型权重比较简单: 在小批量数据集上计算损失函数, 然后利用反向传播算法让权重向正确的方向移动. 而更新超参数比较具有挑战性:
- 计算反馈信号(这组超参数是否得到了一个高性能的模型)的计算代价可能非常的高, 需要在数据集上创建一个新模型并从头开始训练.
- 超参数空间通常由许多离散的决定组成, 既不连续, 也不可微. 因此, 不能在超参数空间中做梯度下降. 必须依赖不使用梯度的优化方法, 而这些方法的效率比梯度下降低很多.  
通常情况下, 随机搜索(随机选择需要评估的超参数, 并重复这一过程)就是最好的解决方案.
还有一种方案Hyperopt(一个用于超参数优化的python库, 内部使用Parzen估计器的树来预测哪组超参数可能会得到好的效果. Hyperas则将Hyperopt和keras模型集成到了一起, 值得一试.  
在进行大规模超参数自动化时, 有一个重要的问题需要牢记, 那就是验证集过拟合. 因为是使用验证集数据计算出一个信号, 然后根据这个信号更新超参数. 所以实际是在验证数据上训练超参数, 很快就会对验证数据过拟合.

### 模型集成
将一系列不同的模型的预测结果汇集到一起, 从而得到更好的预测结果. 模型集成基于假设, 独立训练的不同的良好模型, 他们表现良好可能是因为不同的原因, 每个模型都从略有不同的角度观察数据来作出预测, 得到真相的一部分, 但不是全部真相. 将不同的真相汇集到一起, 可以对数据的描述更加准确.  
将分类器集成有一个更聪明的办法, 加权平均, 权重值在验证数据上学习得到. 通常来说更好的分类器会有更大的权重.  
为了保证集成方法有效, 关键在于分类器的多样性. 如果所有模型的偏差都在同一个方向上, 那么集成模型也会保留同样的偏差. 如果各个模型的偏差在不同方向上, 那么这些偏差会彼此抵消, 集成结果会更加稳定, 更加准确.  
因此, 集成的模型应该尽可能的好, 尽可能的不同. 这意味着, 使用非常不同的架构, 甚至是不同的机器学习方法. 同时, 对相同的网络使用不同的随机初始化多次独立训练, 然后集成是不值得做的.  
将基于树的方法和深度神经网络进行集成在实践中也非常有效. 集成不在于最佳模型有多好, 而在于候选模型的多样性.  
较为成功的集成方法是宽且深的模型类型, 能够结合深度学习与浅层学习. 这种模型联合训练一个深度神经网络和一个大型的线性模型. 对多种模型联合训练, 是实现模型集成的另一种选择.

- 构建高性能的深度卷积神经网络时, 需要使用残差连接, 批标准化和深度可分离卷积. 深度可分离卷积在未来很可能会完全取代普通卷积, 因为深度可分离卷积的表示效率更高.
- 构建深度网络需要选择许多超参数和架构, 这些选择共同决定了模型的性能. 为了找到最佳选择, 目前来看,最好系统的搜索超参数空间. 目前, 这个搜索过程的计算代价还很高, 使用的工具也不是很好, 但Hyperopt和Hyperas这两个库可能会由帮助, 在进行超参数选择时, 一定要小心验证集过拟合.
- 想要在机器学习竞赛中获胜, 或者想要在某项人物上获得最佳结果, 只能通过多个模型的集成来实现. 利用加权平均进行集成通常能够取得足够好的效果. 多样性就是力量, 好的集成方法是将尽可能不同的一组模型集成. 

- 如何将模型构建成层组成的图, 层的重复使用与将模型作为python函数.
- 使用keras回调函数在训练过程中监控模型, 并根据模型状态采取行动.
- Tensorboard可以将指标, 激活直方图甚至嵌入空间可视化.
- 什么是批标准化, 深度可分离网络和残差连接.
- 为什么应该使用超参数优化和模型集成.